In [1]:
# hist comparison
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import os
import importlib


from ATARI.sammy_interface import sammy_interface, sammy_classes, sammy_functions, template_creator

from ATARI.ModelData.particle_pair import Particle_Pair

from ATARI.ModelData.experimental_model import Experimental_Model

from copy import copy

from ATARI.AutoFit import chi2_eliminator_v2
from ATARI.AutoFit import elim_addit_funcs

# for animation
from PIL import Image
import imageio
import glob


In [2]:
%matplotlib widget

In [3]:
settings = {
    'path_to_SAMMY_exe': '/home/fire/SAMMY/sammy/build/install/bin/sammy',
    'path_to_SAMMY_temps': './sammy_temps/',
    'keep_runDIR_SAMMY': True,
    'shell_SAMMY': 'bash',
}

savefolder = './data/'
savefolder = './data_new/'

# creating and saving images
anim_save_dir = savefolder+'anim/'

# Ensure the folder exists
if not os.path.exists(anim_save_dir ):
    os.makedirs(anim_save_dir)

all_histories_data = {
    '0': {'dataset_pkl': 'dataset_sf_33_er[202_227]_chi2allowed_0.pkl',
           'hist_pkl': 'hist_sf_33_er[202_227]_chi2allowed_0.pkl',
           },

    '0_asg_sf_35': {
        'dataset_pkl': 'dataset_sf_35_er[202_227]_chi2allowed_0_allspingr.pkl',
        'hist_pkl': 'hist_sf_35_er[202_227]_chi2allowed_0_allspingr.pkl'
    },

    '0_asg_sf_50':{
        'dataset_pkl': 'dataset_sf_50_er[202_227]_chi2allowed_0_allspingr.pkl',
        'hist_pkl': 'hist_sf_50_er[202_227]_chi2allowed_0_allspingr.pkl'
    },
    
    # start from 35, 15 - start deef fitting from, greedy mode
    '0_asg_sf_35_greedy_sdf_15':{
        'dataset_pkl': 'dataset_sf_35_greedy_True_er[202_227]_chi2allowed_0_allspingr_sdf_15.pkl',
        'hist_pkl': 'hist_sf_35_greedy_True_er[202_227]_chi2allowed_0_allspingr_sdf_15.pkl'
    },
    
    # one spin group used       
    '28': {'dataset_pkl': 'dataset_sf_33_er[202_227]_chi2allowed_28.pkl',
           'hist_pkl': 'hist_sf_33_er[202_227]_chi2allowed_28.pkl',
           },

    '100': {'dataset_pkl': 'dataset_sf_33_er[202_227]_chi2allowed_100.pkl',
           'hist_pkl': 'hist_sf_33_er[202_227]_chi2allowed_100.pkl' },

    '10000': {'dataset_pkl': 'dataset_sf_33_er[202_227]_chi2allowed_10000.pkl',
           'hist_pkl': 'hist_sf_33_er[202_227]_chi2allowed_10000.pkl' },
}


all_histories_data = {
    # greedy with not greedy on window 10 eV
    '0_asg_sf_24_sdf_10_ng':{
        'dataset_pkl': 'dataset_sf_24_greedy_False_er[201_210]_chi2allowed_0_allspingr_sdf_10.pkl',
        'hist_pkl': 'hist_sf_24_greedy_False_er[201_210]_chi2allowed_0_allspingr_sdf_10.pkl'
    },

    '0_asg_sf_24_sdf_10_g':{
        'dataset_pkl': 'dataset_sf_24_greedy_True_er[201_210]_chi2allowed_0_allspingr_sdf_10.pkl',
        'hist_pkl': 'hist_sf_24_greedy_True_er[201_210]_chi2allowed_0_allspingr_sdf_10.pkl'
    },

    '0_asg_sf_24_sdf_100_g':{
        'dataset_pkl': 'dataset_sf_24_greedy_True_er[201_210]_chi2allowed_0_allspingr_sdf_100.pkl',
        'hist_pkl': 'hist_sf_24_greedy_True_er[201_210]_chi2allowed_0_allspingr_sdf_100.pkl'
    },

        # test on other energy window
    # start from 30, 25 - start deef fitting from, greedy mode
    '0_asg_sf_30_greedy_sdf_25':{
        'dataset_pkl': 'dataset_sf_30_greedy_True_er[227_252]_chi2allowed_0_allspingr_sdf_25.pkl',
        'hist_pkl': 'hist_sf_30_greedy_True_er[227_252]_chi2allowed_0_allspingr_sdf_25.pkl'
    },
    # start from 30, 15 - start deef fitting from, greedy mode
    '0_asg_sf_30_greedy_sdf_15':{
        'dataset_pkl': 'dataset_sf_30_greedy_True_er[227_252]_chi2allowed_0_allspingr_sdf_15.pkl',
        'hist_pkl': 'hist_sf_30_greedy_True_er[227_252]_chi2allowed_0_allspingr_sdf_15.pkl'
    },
    
}

all_histories_data = {
    '100_asg_sf_15_greedy_sdf_14':{
        'dataset_pkl': 'dataset_sf_15_greedy_True_er[277_302]_chi2allowed_100_sdf_14.pkl',
        'hist_pkl': 'hist_sf_15_greedy_True_er[277_302]_chi2allowed_100_sdf_14.pkl'
    },
    '50_asg_sf_15_greedy_sdf_14':{
        'dataset_pkl': 'dataset_sf_15_greedy_True_er[277_302]_chi2allowed_50_sdf_14.pkl',
        'hist_pkl': 'hist_sf_15_greedy_True_er[277_302]_chi2allowed_50_sdf_14.pkl'
    },
    }

fig_size_data = (9,8)
fig_size_hist = (4,8)

colors = ["C1", "C2", "C3", "C4", "C5", "C6", "C7"]


In [4]:
chi2_labels = []
allexp_data = {}

for key, value in all_histories_data.items():

    chi2_allowed = key
    dataset_pkl = value['dataset_pkl']
    hist_pkl = value['hist_pkl']

    energy_range, datasets, covariance_data, experiments, true_chars, Ta_pair, hist, elim_OPTS_used = elim_addit_funcs.load_all(savefolder=savefolder,
                          hist_pkl_name=hist_pkl,
                          dataset_pkl_name=dataset_pkl)
    
    allexp_data[f'{key}'] = {
        'energy_range': energy_range,
        'datasets': datasets,
        'experiments': experiments,
        'covariance_data': covariance_data,
        'true_chars': true_chars,
        'Ta_pair': Ta_pair,
        'hist': hist,
        'elim_opts': elim_OPTS_used
    }

# print(allexp_data['28']['hist'].elimination_history[30]['total_time'])

In [5]:
# model1_key = 0
# model2_key = '0_asg_sf_35'

# model2_key = '0_asg_sf_50'
# model1_key = '0_asg_sf_35'

model1_key = '0_asg_sf_24_sdf_10_ng'
model2_key = '0_asg_sf_24_sdf_10_g'
model3_key = '0_asg_sf_24_sdf_100_g'

#compare_keys = [f'{model1_key}', f'{model2_key}', f'{model3_key}']

# model1_key = '0_asg_sf_35'
# model2_key = '0_asg_sf_35_greedy_sdf_15'

model1_key = '0_asg_sf_30_greedy_sdf_25'
model2_key = '0_asg_sf_30_greedy_sdf_15'

model1_key = '100_asg_sf_15_greedy_sdf_14'
model2_key = '50_asg_sf_15_greedy_sdf_14'

compare_keys = [f'{model1_key}', f'{model2_key}']

comp_level = 10

settings = {
    'path_to_SAMMY_exe': '/home/fire/SAMMY/sammy/build/install/bin/sammy',
    'path_to_SAMMY_temps': './sammy_temps/',
    'keep_runDIR_SAMMY': True,
    'shell_SAMMY': 'bash',
}

In [6]:
# make a solution comparison table

print('Elim opts used:')
print(allexp_data[f'{model1_key}']['elim_opts'])

table_1 = elim_addit_funcs.create_solutions_comparison_table_from_hist(hist = allexp_data[f'{model1_key}']['hist'],
                                                Ta_pair = allexp_data[f'{model1_key}']['Ta_pair'],
                     datasets =  allexp_data[f'{model1_key}']['datasets'],
                     experiments =  allexp_data[f'{model1_key}']['experiments'],
                     covariance_data =  allexp_data[f'{model1_key}']['covariance_data'],
                     true_chars = allexp_data[f'{model1_key}']['true_chars'],
                     settings=settings,
                     energy_grid_2_compare_on = allexp_data[f'{model1_key}']['energy_range'])


Elim opts used:
LevMarV0_priorpassed: 0.01
chi2_allowed: 100
deep_fit_max_iter: 30
deep_fit_step_thr: 0.001
fixed_resonances_df:         E       Gg    Gn1  varyE  varyGg  varyGn1  J_ID
0  273.74  64.9999  97.94      0       1        1   1.0
6  304.07  64.9999  41.95      0       1        1   2.0
greedy_mode: True
interm_fit_max_iter: 10
interm_fit_step_thr: 0.01
start_deep_fit_from: 14
start_fudge_for_deep_stage: 0.05
stop_at_chi2_thr: False
use_spin_shuffling: False

Completed Job: 1
Warning - using precalculated chi2 value
Using cov data for chi2 calc
Warning - using precalculated chi2 value
Using cov data for chi2 calc
Warning - using precalculated chi2 value
Using cov data for chi2 calc
Warning - using precalculated chi2 value
Using cov data for chi2 calc
Warning - using precalculated chi2 value
Using cov data for chi2 calc
Completed Job: 1
Warning - using precalculated chi2 value
Using cov data for chi2 calc
Warning - using precalculated chi2 value
Using cov data for chi2 calc
War

In [7]:
# for step in allexp_data[f'{model1_key}']['hist'].elimination_history:
#     print(step)

In [9]:
table_1

,AT,N_res,N_res_joint_LL,passed,sum_chi2,chi2_s,SSE,sum_NLLW,AICc,BIC,delta_AICc_best,delta_BIC_best,delta_chi2_prev,delta_chi2_best
0,False,16,-1416.792837,True,698.7505,1.068426,137.524621,60.034510,801.954175,1013.339303,37.696584,80.798701,0.0000,17.2059
1,False,15,-1416.792837,True,702.1951,1.068790,143.141431,57.492396,798.506076,997.122103,34.248485,64.581501,3.4446,20.6505
2,False,14,-1416.792837,True,704.2824,1.067095,143.000918,54.095973,793.763432,979.547603,29.505841,47.007001,2.0873,22.7378
3,False,13,-1416.792837,True,681.5446,1.027971,104.502482,46.297582,764.257591,937.148003,0.000000,4.607400,-22.7378,0.0000
4,False,12,-1416.792837,True,701.6024,1.053457,70.212005,41.336538,777.608415,937.544003,13.350824,5.003400,20.0578,20.0578
5,False,11,-1416.792837,True,716.2608,1.070644,67.361375,35.707959,785.620081,932.540602,21.362491,0.000000,14.6584,34.7162
6,False,10,-1416.792837,True,768.2336,1.143205,41.166076,31.282238,831.005582,964.851602,66.747991,32.311000,51.9728,86.6890
7,False,9,-715.660849,True,764.6365,1.132795,40.598775,24.963708,820.879823,941.592702,56.622233,9.052100,-3.5971,83.0919
8,False,8,-713.546316,True,784.8133,1.157542,27.622433,20.026467,834.585826,942.107702,70.328235,9.567099,20.1768,103.2687
9,False,7,-711.602872,True,821.9025,1.206905,8.880082,15.060063,865.261324,959.535101,101.003733,26.994499,37.0892,140.3579


In [ ]:
fig = elim_addit_funcs.plot_history(allexp_data = allexp_data, 
                                    show_keys = compare_keys, 
                                    settings = settings,
                   fig_size = fig_size_hist, 
                   max_level = comp_level, 
                   title = 'Steps tracking')

fig.show()

In [ ]:
# # times for elimination - for each level and cumulatively

# levels_times = elim_addit_funcs.get_level_times(allexp_data: dict, 
#                  show_keys: list,
#                  settings: dict, 
#                  fig_size: tuple = (6, 10), 
#                  max_level: int = None,
#                  title : str = '')

In [ ]:

model1 = allexp_data[f'{model1_key}']['hist'].elimination_history[comp_level]['selected_ladder_chars']
model2 = allexp_data[f'{model2_key}']['hist'].elimination_history[comp_level]['selected_ladder_chars']

fig = elim_addit_funcs.plot_datafits(datasets, experiments, 
    fits=model1.pw_post, fits_chi2=model1.chi2_post, f_model_name=f'{model1_key}', f_color='red',
    priors=model2.pw_post, priors_chi2=model2.chi2_post, pr_model_name=f'{model2_key}', pr_color='black',

    true=true_chars.pw_post, true_chi2 = true_chars.chi2_post, t_model_name ='JEFF (true, post)', t_color='green',
        true_pars = true_chars.par_post,
        fit_pars = model1.par_post, 
        prior_pars = model2.par_post,

    title = f'Models Comparison. Step {comp_level}',
    show_spingroups = False,
    fig_size = fig_size_data
    )

fig.show()

In [ ]:
# cycle of images creation

out_images = []

for step in allexp_data[f'{model1_key}']['hist'].elimination_history:
    print(step)

    model1 = allexp_data[f'{model1_key}']['hist'].elimination_history[step]['selected_ladder_chars']
    model2 = allexp_data[f'{model2_key}']['hist'].elimination_history[step]['selected_ladder_chars']

    # paths to save figures
    model_comp_path = anim_save_dir + f'model_comp_{model1_key}_{model2_key}_step_{step}.png'
    chi2_hist_path = anim_save_dir + f'chi2_hist_{model1_key}_{model2_key}_step_{step}.png'
    combined_image_path = anim_save_dir + f'combined_{model1_key}_{model2_key}_step_{step}.png'

    fig = elim_addit_funcs.plot_datafits(datasets, experiments, 
        fits=model1.pw_post, fits_chi2=model1.chi2_post, f_model_name=f'{model1_key}', f_color='red',
        priors=model2.pw_post, priors_chi2=model2.chi2_post, pr_model_name=f'{model2_key}', pr_color='black',

        true=true_chars.pw_post, true_chi2 = true_chars.chi2_post, t_model_name ='JEFF (true, post)', t_color='green',
            true_pars = true_chars.par_post,
            fit_pars = model1.par_post, 
            prior_pars = model2.par_post,

        title = f'Models Comparison. Step {step}',
        show_spingroups = False,
        fig_size = fig_size_data
        )
    
    fig.savefig(fname = model_comp_path)

    # get the plots of history for this step
    fig_hist = elim_addit_funcs.plot_history(allexp_data = allexp_data,
                            show_keys = compare_keys,
                            settings=settings,
                            fig_size = fig_size_hist,
                            max_level = step,
                            title='Steps tracking',
                            folder_to_save=anim_save_dir)
    
    fig_hist.savefig(fname = chi2_hist_path)

    # combine both images into one left+right (left - model_comp_step, right - chi2_hist_step_)
    image1 = Image.open(model_comp_path)
    image2 = Image.open(chi2_hist_path)

    # Determine the total width and max height of the combined image
    total_width = image1.width + image2.width
    max_height = max(image1.height, image2.height)

    # Create a new blank image with the correct size
    combined_image = Image.new('RGB', (total_width, max_height))

    # Paste the two images into the combined image
    combined_image.paste(image1, (0, 0))
    combined_image.paste(image2, (image1.width, 0))

    # Save the combined image
    combined_image.save(combined_image_path)
    out_images.append(combined_image_path)
    print(f'saved {combined_image_path}')

    # delete intermediate images
    # del model_comp_path and chi2_hist_path
    os.remove(model_comp_path)
    os.remove(chi2_hist_path)

In [ ]:
def create_animation(pattern: str,  
                     output_name: str,
                     anim_save_dir: str,
                     out_images: list = [],
                     additional_start_images: list =[], 
                     additional_end_images: list =[], 
                     start_duration: int = 3,
                     intermediate_duration: int = 2,
                     end_duration: int = 2):
    
    """Create an animation based on the specified parameters."""
    print('pattern', pattern)

    # If we don't have any data on out_images, parse the directory
    if len(out_images) == 0:
        path_to_search = os.path.join(anim_save_dir, pattern)

        matched_files = glob.glob(path_to_search)

        #matched_files.sort(key=lambda f: int(os.path.basename(f).split('_')[2]), reverse=True)
        matched_files.sort(key=lambda f: int(os.path.basename(f).split('step_')[-1].split('.png')[0]), reverse=True)

        out_images = matched_files
    
    # Convert durations to milliseconds
    start_duration_ms = start_duration * 1000
    intermediate_duration_ms = intermediate_duration * 1000
    end_duration_ms = end_duration * 1000
    
    # Combine all images in the required order
    all_images = additional_start_images + out_images + additional_end_images
    
    # Print image sizes for verification (optional)
    for img_path in all_images:
        with Image.open(img_path) as img:
            print(f"{os.path.basename(img_path)} - Size: {img.size}")
    
    # Create a list of images from the combined file list
    images = [Image.open(f) for f in all_images]
    
    # Output path for the animated GIF
    output_path = os.path.join(anim_save_dir, output_name)
    
    # Create a list of durations corresponding to each image
    durations = ([start_duration_ms] * len(additional_start_images) + 
                 [intermediate_duration_ms] * len(out_images) + 
                 [end_duration_ms] * len(additional_end_images))
    
    # Save the animated GIF using imageio
    imageio.mimsave(output_path, images, duration=durations) 
    
    print(f"Animated GIF saved to {output_path}")
    return output_path  # Return the output path for further use if needed

#######
#######
#######
# combined plots

pattern = f'combined_{model1_key}_{model2_key}_step_*.png'
output_name = f'output_gif_{compare_keys[0]}_{compare_keys[1]}.gif'

# Attempt to use out_images if already defined

out_images = []  # or some default value or handling

anim_path_name = create_animation(pattern = pattern,  
                     output_name = output_name,
                     anim_save_dir = anim_save_dir,
                     out_images = out_images,
                     additional_start_images =[], 
                     additional_end_images =[], 
                     start_duration = 3,
                     intermediate_duration = 2,
                     end_duration = 2)

# xs for the first model
pattern = f'xs_{model1_key}_step_*.png'
output_name = f'output_xs_{compare_keys[0]}.gif'

out_images = []  # or some default value or handling

anim_path_name = create_animation(pattern = pattern,  
                     output_name = output_name,
                     anim_save_dir = anim_save_dir,
                     out_images = out_images,
                     additional_start_images =[], 
                     additional_end_images =[], 
                     start_duration = 3,
                     intermediate_duration = 2,
                     end_duration = 2)

pattern = f'xs_{compare_keys[1]}_step_*.png'
output_name = f'output_xs_{compare_keys[1]}.gif'
out_images = []  # or some default value or handling

anim_path_name = create_animation(pattern = pattern,  
                     output_name = output_name,
                     anim_save_dir = anim_save_dir,
                     out_images = out_images,
                     additional_start_images =[], 
                     additional_end_images =[], 
                     start_duration = 3,
                     intermediate_duration = 2,
                     end_duration = 2)


In [ ]:
# out_images